# ETF DV01对冲（以富国中债7-10年政金债ETF为例）

本项目基于 Streamlit 构建轻量化可视化工具，旨在服务于中国利率债ETF的主动量化做市策略。当前核心研究对象为 **富国中债7-10年政金债ETF（511520.SH）**，并使用 **10年期国债期货（T2509合约）** 进行 DV01 中性对冲计算与建议。

---

## 国债期货对冲基本原理

在利率债ETF的做市过程中，做市商为防止利率波动导致的仓位亏损，常使用国债期货进行 DV01 中性对冲。核心逻辑如下：

1. **ETF 持仓 DV01 = ETF市值 × 久期 × 0.0001**
   - 例如：持有 10,000 份 ETF，价格 114.6，久期 7.5 → DV01 ≈ 859.5 元/bp

2. **期货 DV01 = (CTD券DV01 ÷ 转换因子) × 10000**
   - CTD券DV01：由其久期估算或查中债估值；
   - 转换因子：由中金所发布（如 T2509 = 0.85）

3. **推荐对冲手数 = ETF_DV01 ÷ 期货_DV01**
   - 精确匹配两者利率敏感度，使净 DV01 接近于 0

通过动态计算推荐期货头寸，做市商可在挂盘同时实现对冲保护

## 为什么选择 511520 作为研究对象？

中国利率债ETF根据所追踪指数的久期结构，大致可以分为以下几类：

| ETF名称 | 基金公司 | 跟踪指数 | 指数久期 | 近一年波动率 |
|---------|----------|----------|----------|--------------|
| 国泰上证5年期国债ETF（511010） | 国泰基金 | 上证5年期国债指数 | ≈4.4年 | ~1.1% |
| 国泰上证10年期国债ETF（511260） | 国泰基金 | 上证10年期国债指数 | ≈7.6年 | ~1.4% |
| 平安中证5-10年国债活跃券ETF（511020） | 平安基金 | 中证5-10年活跃券指数 | ≈5.5年 | ~1.6% |
| 富国中债7-10年政金债ETF（511520） | 富国基金 | 中债-7-10年政金债指数 | ≈7.5年 | ~1.5% |
| 鹏扬30年期国债ETF（511090） | 鹏扬基金 | 中债30年期国债指数 | ≈23年 | >4% |

在策略设计上：

- **短久期ETF**（如511010）点差极窄，更适合作为低风险配置；
- **超长久期ETF**（如511090）波动大但难以对冲；
- **富国511520 处于中间位置**，波动适中、流动性良好，是利率对冲的理想对象；
- 搭配**10年期国债期货（T2509）**，久期匹配，利于构建 DV01 中性对冲结构。


---

## 项目目标

- 实时输入ETF价格、持仓份额，估算组合DV01利率敞口；
- 根据国债期货DV01自动匹配对冲手数；
- 可视化展示ETF走势、推荐对冲结构、净敞口变化；
- 可扩展预测模块（GPT情绪建议、技术指标等）；

---

## 文件夹结构

```
etf_dv01_hedging_tool/
├── app/                         # 主程序目录（Streamlit）
│   ├── app.py                   # 主页面入口
│   └── components/              # 子功能模块（如对冲算法、图表组件）
│
├── data/                        # 存储ETF和期货历史数据
│   ├── etf_511520.csv
│   ├── futures_T.csv
│
├── notebooks/                   # Jupyter Notebook用于模型原型验证
│   └── dv01_calculation.ipynb
│
├── scripts/                     # 拉取/清洗数据的脚本（如AkShare）
│   └── fetch_data_akshare.py
│
├── utils/                       # 公共函数模块（DV01计算、预测封装等）
│   └── dv01_calc.py
│
├── requirements.txt             # 项目所需依赖包列表
├── README.md                    # 项目说明文档（本文件）
└── ETF_DV01_Hedging_Tool_项目说明.md   

---

## 合并后的示例数据（前5行）

| date       | close_etf | volume_etf | turnover | pct_change | close_fut | volume_fut | open_interest |
|------------|------------|-------------|------------|-------------|------------|-------------|----------------|
| 2024-12-16 | 114.655    | 705557      | 8.08亿     | 0.40%       | 108.545    | 1181        | 237            |
| 2024-12-17 | 114.448    | 790942      | 9.04亿     | -0.18%      | 108.460    | 1234        | 1048           |
| 2024-12-18 | 114.243    | 356201      | 4.07亿     | -0.18%      | 108.340    | 942         | 1242           |
| 2024-12-19 | 114.251    | 615158      | 7.03亿     | 0.01%       | 108.380    | 379         | 1325           |
| 2024-12-20 | 114.678    | 656408      | 7.52亿     | 0.37%       | 108.770    | 345         | 1512           |

```
## 字段说明

| 字段名         | 含义           | 用途（对冲分析中）                             |
|----------------|----------------|-----------------------------------------------|
| `date`         | 交易日期       | 用于对齐 ETF 与 T 期货                        |
| `close_etf`    | ETF 收盘价     | 用于估算 ETF 总市值和 DV01                    |
| `volume_etf`   | ETF 成交量     | 辅助判断流动性/日间活跃度                    |
| `turnover`     | ETF 成交额     | 辅助估算大致日成交资金流                      |
| `pct_change`   | ETF 涨跌幅     | 估算波动程度，可配合预测模型使用              |
| `close_fut`    | T2509 收盘价   | 用于估算期货 DV01、对冲计算                   |
| `volume_fut`   | 期货成交量     | 判断期货活跃度，是否适合执行对冲操作          |
| `open_interest`| 期货持仓量     | 反映多空博弈结构/市场深度，辅助对冲信心判断  |

---

## 使用方式（本地运行）

1. 克隆项目：
```bash
git clone https://github.com/yourname/etf_dv01_hedging_tool.git
cd etf_dv01_hedging_tool
```

2. 创建虚拟环境（推荐）并安装依赖：
```bash
python -m venv venv
source venv/bin/activate  # Windows用户使用 venv\Scripts\activate
pip install -r requirements.txt
```

3. 运行主程序：
```bash
streamlit run app/app.py
```

---

## 模块简述

| 模块 | 描述 |
|------|------|
| DV01估算 | ETF价格 × 份额 × 久期 × 0.0001，支持自定义输入 |
| 期货DV01 | 默认设置为T合约每手约490元，可调整 |
| 对冲建议 | 自动计算推荐空头手数，展示敞口是否中性 |
| 可视化 | 支持图表（ETF走势、净敞口、预测建议等） |
| 预测模块 | 可对接GPT或技术指标辅助判断走势（可选） |

---

## 后续扩展方向

- Wind / Choice 实时行情接入
- 多ETF组合同时监控
- 添加止盈止损、自动调仓功能
- 对接交易接口实现交易闭环

---


